In [45]:
import scipy.io
import scipy.io.wavfile
import sounddevice as sd

In [55]:
def play_from_filepath(fp, name='master', framerate=16000, start_in_s=0):
    sound = scipy.io.loadmat(fp)
    play_from_array(sound['earSout'], name, framerate, start_in_s=start_in_s)
    
    
def play_from_array(sound, name, framerate, start_in_s=0):
    start_frame = int(start_in_s * framerate)
    sound_arr = sound[start_frame:]
    # somehow try to invert the second axis - may resolve the feeling that the events happen on the left side
    #sound_arr = np.swapaxes(np.swapaxes(sound_arr, 0, sound_arr)[::-1], 0, 1)
    scipy.io.wavfile.write('demo_sound_{}_start{}s.wav'.format(name, str(start_in_s)), rate=framerate, data=sound_arr)
    sd.play(sound_arr, framerate)

In [52]:
fp = "/Users/Heiner/repos/binaural_audition/common/data/data_creation/generalSoundsNI.phone.TelephoneElectron+6037_36_3.wav.mat"
play_from_filepath(fp, 'master', start_in_s=0)

In [53]:
sd.stop()

In [56]:
# distractor 1

fp = '/Users/Heiner/repos/binaural_audition/common/data/data_creation/distractor1_no2_CloseOutsideMic2.wav'
framerate, array = scipy.io.wavfile.read(fp)
play_from_array(array, 'distractor1_no2', framerate, start_in_s=25-1.9219)

fp = '/Users/Heiner/repos/binaural_audition/common/data/data_creation/distractor1_no3_BAG-PAPER_GEN-HDF-02778.wav'
framerate, array = scipy.io.wavfile.read(fp)
play_from_array(array, 'distractor1_no3', framerate, start_in_s=0)

fp = '/Users/Heiner/repos/binaural_audition/common/data/data_creation/distractor1_no4_ScissorPaper_DIGIF5-88.wav'
framerate, array = scipy.io.wavfile.read(fp)
play_from_array(array, 'distractor1_no4', framerate, start_in_s=0)

# distractor 2

fp = '/Users/Heiner/repos/binaural_audition/common/data/data_creation/distractor2_no3_CartoonFootsteps+4003_46_1.wav'
framerate, array = scipy.io.wavfile.read(fp)
play_from_array(array, 'distractor2_no3', framerate, start_in_s=25-12.0458)

/Users/Heiner/anaconda3/lib/python3.6/site-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


PortAudioError: Error opening OutputStream: Internal PortAudio error [PaErrorCode -9986]